In [6]:
# import Pkg; Pkg.add("CSV")

  Resolving package versions...
  Installed CSV ─ v0.8.2
Updating `~/Development/ATIS3_2020/Project.toml`
  [336ed68f] + CSV v0.8.2
Updating `~/Development/ATIS3_2020/Manifest.toml`
  [336ed68f] + CSV v0.8.2
  [91c51154] + SentinelArrays v1.2.16


In [4]:
using Pkg
Pkg.activate("/Users/demircanm/Development/ATIS3_2020") # change path 
Pkg.instantiate()

 Activating environment at `~/Development/ATIS3_2020/Project.toml`


In [7]:
using JuMP, LinearAlgebra, Plots, StatsPlots, CPLEX, Statistics, CSV, DataFrames

┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1278
┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1278


In [8]:
#read scenario data
scenarios = CSV.read("/Users/demircanm/sciebo/IS3/Research_Projects/ATIS3-Group/01 Data/scenarios_multinomial.csv", DataFrame)

SOC = 0.33
hexagons = names(scenarios)[1:length(names(scenarios))-1] # vector of hexagons (strings)
hours = 4 # duration for charging
car_kWh = 30 # capacity of each car

#demand mxn (m=hexagon, n=scenario) in kWh summe über car*max_kWh*(1-SOC)
demand_cars = transpose(convert(Matrix,scenarios[:,hexagons]))
demand_kWh = demand_cars.*car_kWh.*(1-SOC)

probability = transpose(scenarios.probs) # probability for each scenario

P = [3, 7, 11] # different charger types in kW
charger_cost = [1000, 800, 500] # investment cost for charger 3 kW, 7 kW und 11 kW in €/kW
opportunity_cost = 5000 # cost of demand not served


5000

In [12]:
probability

1×6 Transpose{Float64,Array{Float64,1}}:
 0.163  0.243  0.154  0.156  0.127  0.157

In [187]:
function charger_siting(SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, probability, P,
        charger_cost, opportunity_cost)
    
    # Define Sets
    Γ = Array{Int}(1:size(hexagons,1)) # Set with hexagons
    Σ = Array{Int}(1:size(P,1)) # Set with charger types 
    Ω = Array{Int}(1:size(probability,2)) # Set with scenarios

    # Initialize model    
    m = Model(CPLEX.Optimizer)
    


    # Define variables
    @variable(m, 0 <= x_charger[i=Γ, j=Σ], Int) # number of specific charger type in each hexagon
    @variable(m, 0 <= E_supplied[i=Γ, j=Σ, k=Ω]) # supplied energy in each hexagon kWh
    @variable(m, 0 <= E_not_supplied[i=Γ, k=Ω]) # demand not covered kWh
    @variable(m, 0 <= cars_supplied[i=Γ, j=Σ, k=Ω], Int)
    @variable(m, 0 <= cars_not_supplied[i=Γ, j=Σ, k=Ω], Int)
    
    # CVAR
    @variable(m, cVAR) 
    @variable(m, 0 <= s[k=Ω])
    
    β = 1.0
    α = 0.95


    # Objective function 
    @expression(m, invest_cost[i=Γ,j=Σ], charger_cost[j].*x_charger[i,j].*P[j]) # first stage investment cost
    @expression(m, charging_cost[i=Γ,k=Ω], opportunity_cost.*E_not_supplied[i,k].*probability[k])
    
    @objective(m, Max, (1-β)*(-1)*(sum(invest_cost)+sum(charging_cost)) + (β*(cVAR-(1/(1-α) * dot(probability, s)))) )

    # Constraints
    # Eq. Balance equation: Demand in each hexagon has to equal demand. Loss of load is possible
    @constraint(m, eq_balance[i=Γ,k=Ω], sum(E_supplied[i,j,k] for j in Σ) + E_not_supplied[i,k] == demand_kWh[i,k])

    # Eq. energy limit: rated_power*chargingtime must not exeed energy
    @constraint(m, eq_cap[i=Γ,j=Σ,k=Ω], E_supplied[i,j,k] <= x_charger[i,j].*P[j].*hours)

    # Eq. supplied cars and not supplied cars have to equal amount of cars in each hexagon
    @constraint(m, eq_cars_supplied[i=Γ,j=Σ,k=Ω], sum(cars_supplied[i,j,k] for j in Σ) 
                                                + sum(cars_not_supplied[i,j,k] for j in Σ)
                                                == demand_cars[i,k])

    # Eq. maximum one car per charging station
    @constraint(m, eq_max_cars[i=Γ,j=Σ,k=Ω], cars_supplied[i,j,k] <= x_charger[i,j])

    # Eq. 
    @constraint(m, eq_max_supply[i=Γ,j=Σ,k=Ω], E_supplied[i,j,k] <= cars_supplied[i,j,k].*car_kWh.*(1-SOC))

    # CVAR Constraint
    @constraint(m, cvar_constraint[k=Ω], cVAR + (sum(charger_cost[j].*x_charger[i,j].*P[j] for j in Σ for i in Γ) + sum(opportunity_cost.*E_not_supplied[i,k] for i in Γ)) <= s[k])


    
    # Run optimization
    optimize!(m)

    # Evaluate resuluts
    obj = objective_value(m)
    charger_in_hex = value.(x_charger)
    cond_val_risk = value.(cVAR)

    return charger_in_hex, x_charger, E_not_supplied, E_supplied, cars_not_supplied, cars_supplied, cond_val_risk, obj
end


charger_siting (generic function with 1 method)

In [188]:
charger_in_hex, x_charger, E_not_supplied, E_supplied, cars_not_supplied, cars_supplied, c_var, obj = charger_siting(
    SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, probability, P, charger_cost, opportunity_cost)



println(charger_in_hex)

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
Tried aggregator 1 time.
MIP Presolve eliminated 3298 rows and 2779 columns.
MIP Presolve modified 162 coefficients.
Reduced MIP has 4274 rows, 3339 columns, and 11856 nonzeros.
Reduced MIP has 296 binaries, 1484 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (9.00 ticks)
Found incumbent of value -1.1043442e+09 after 0.02 sec. (12.82 ticks)
Probing time = 0.00 sec. (0.24 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 4274 rows, 3339 columns, and 11856 nonzeros.
Reduced MIP has 296 binaries, 1484 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (7.49 ticks)
Probing time = 0.00 sec. (0.27 ticks)
Clique table members: 130.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.03 sec. (16.56 ticks)

        Nodes                                         Cuts/
   Node  Le

    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  88, 89, 90, 91, 92, 93, 94, 95, 96, 97]
    Dimension 2, [1, 2, 3]
And data, a 97×3 Array{Float64,2}:
  0.0                 0.0                     1.0
  0.0                 0.0                     0.0
  0.0                 0.0                     0.0
 15.0                 1.1111049302847675e-9   0.0
  0.0                 0.0                     0.0
 10.0                 0.0                     0.0
  0.0                 0.0                     0.0
  0.0                 0.0                     0.0
  0.0                 0.0                     0.0
 10.0                 0.0                     0.0
  0.0                 0.0                     1.0
  0.0                 0.0                     1.0
  5.0                 0.0                     1.0
  7.0                 0.0                     1.0
  0.0                 0.0                     0.0
  5.0                 0.0                     0.0
 42.0                 0.0                     

In [190]:
c_var

-3.790499999996766e6

In [180]:
-3787079/ Null CVAR

LoadError: syntax: extra token "CVAR" after end of expression

In [ ]:
# CVAR (alpha 0.7)
-3790500/-3790500